# Voting Analysis
## Group 6 - Alek Peters, John Valderama, Heather Novak

------------------------------------------------------------------------

## Q: Could the majority of the popular vote for U.S. President in 2012 have been cast for Romney rather than Obama because of voter fraud?

## Take a Look at the Data

In [1]:
import pandas as pd
import requests

In [2]:
r = requests.get('https://publicinterestlegal.org/county-list', headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})

In [3]:
regVoters = pd.read_html(r.text, header=0)[0]

**Data Dictionary**
* COUNTY(string): The name of the county
* STATE(string): The name of the state
* REGISTRATION RATE(float): The amount percentage the population that are registered to vote.

In [4]:
elecdf = pd.read_excel('http://image.guardian.co.uk/sys-files/Guardian/documents/2012/11/14/US_elect_county.xls')
elecdf.rename(columns={'%':'Obama%', '%.1':'Romney%'}, inplace=True)

WARNING *** file size (3825153) not 512 + multiple of sector size (512)


#### Q: What data fields might be useful in answering the research question and how could you use them?
##### A: We could use the STATE and REGISTRATION RATE fields from the 'county list' data to determine the states with excess voters. We can combine this data with the 'US Elect County' data, specifically the fields State Postal (Which we can convert to match STATE from the other data), the Obama % rate, and the Romney % rate. We would use these fields to determine what the outcome was in the states that were determined to have excess votes.

In [5]:
# Make the state column the same format for both dataframes
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

# replace the state postal name with the full state names
elecdf['State Postal'].replace(states, inplace=True)

**Data Dictionary**

* State (string): State Name
* County Name(string): Name of county
* FIPS(int): FIPS code for the state
* Obama vote(int): Number of votes for Obama
* Obama%(float): Percentage of voters that voted for Obama
* Romney vote: Number of votes for Romneu
* Romney%: The percentage of voters that voted for Romney

#### Q: How is the data formatted (what type of files, string versus numerical data)?
##### A: The data from the excel spreadsheet is formatted as objects except for FIPS which is int64. We can redefine these types to be numeric/int.
##### Registration rate is set as an object, but needs to be a float value. That's due to the "%". We can convert it to a float by removing that and coercing the column to float.

In [6]:
#convert REGISTRATION RATE to float values
regVoters['REGISTRATION RATE'] = regVoters['REGISTRATION RATE'].str.replace('%', '').astype('float')

#### Q: Did your team find any missing data?
##### A: We did not find any missing data. We interpreted 'missing data' as data with a NaN value or empty strings.

#### Q: Did your team find any data that could be erroneous?
##### A: I don't think that the county or zip code matter unless we wanted to really drill down the data. For a high level response to the question asked, I think that state is fine since the votes for President are determined per state and not per county or district. The Excel file had a second tab with all of the data in it which was unused. This data included third party or independent candidates and went into more detail.

#### Q: What unusual facets of the data should the analysis take into account?
##### A: Not that this is 'unusual' but something to consider is that there is a possibility that there were fraudulent votes for a third party which wouldn't be shown in this dataset, but could be found in the FULL DATA tab in the excel workbook for 'US Elect County'

## Analysis Design

#### Q: Give a formula for a computation your team would use for this analysis. Describe your computation specifically and how you assess it relative to the criteria in the video.
##### A: Votes*(registrationrate-100)-ObamaMarginofVictory = Potential Impact of Fraudulent Votes on Election Outcome

#### Q: Explain why this computation makes sense, in a few short sentences.
##### A: This would calculate the percentage of fraudulent votes for Obama based on total votes across the country. This computation makes sense to evaluate the potential impact of fraudelent votes on the outcome of the election by comparing the percentage of fraudulent votes with the margin of victory in the election.

#### Q: What data fields would your team use to tie together the two data sources?
##### A: As stated above, we could use the STATE and REGISTRATION RATE fields from the 'county list' data to determine the states with excess voters. We can combine this data with the 'US Elect County' data, specifically the fields State Postal (Which we can convert to match STATE from the other data), the Obama % rate, and the Romney % rate. We would use these fields to determine what the outcome was in the states that were determined to have excess votes.

In [8]:
regVoters['COUNTY'] = regVoters['COUNTY'].str.replace(' County', '')

In [9]:
# merge dataframes on state & county
fraudCounties = pd.merge(regVoters, elecdf, left_on=['STATE', 'COUNTY'], right_on=['State Postal', 'County Name'])

In [11]:
fraudCounties.head()

,COUNTY,STATE,REGISTRATION RATE,State Postal,County Name,FIPS,Obama vote,Obama%,Romney vote,Romney%
0,Lowndes,Alabama,127.0,Alabama,Lowndes,1085,5747,76.4,1754,23.3
1,Perry,Alabama,113.0,Alabama,Perry,1105,4536,74.8,1504,24.8
2,Greene,Alabama,112.0,Alabama,Greene,1063,4514,84.7,799,15.0
3,Macon,Alabama,112.0,Alabama,Macon,1087,9045,87.1,1322,12.7
4,Wilcox,Alabama,110.0,Alabama,Wilcox,1131,4867,74.3,1676,25.6


#### Q: What software might you use in tying the data sources together?
##### A: You could use Python, R, or Excel to tie the data together.

#### Q: What issues do you see in correlating those fields due to, for example, differences in formatting?
##### A: Fields need to be numeric for graphs, charts, or calculations. If any of the fields are string values because of dollar signs, commas, etc., they will need to be stripped and converted before proceeding. The column values need to match in order for the dataframes to merge together correctly. For example the counties were named differently, so Lowndes and Lowndes County wouldn't be able to merge since it's seen as two different counties. This was demonstrated in the above code.

## Propose a Better Question

#### Q: What improved research question would your team suggest?
##### A: Which states experienced voter fraud and which candidate won those states? Additionally, did the fraudulent votes matter to the outcome (was the margin of victory equal to or less than the number of fraudulent votes)?

#### Q: Why is your question an improvement on the original question?
##### A: This question is an improvement to the original question because it takes into account the election process. The popular vote is important, but the electoral college determines the presidency. Therefore, looking at voter fraud by state and determining if fraud could have changed the state outcome is a more important question.